In [27]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [28]:
# Make a url list
url_base = 'https://hoopshype.com/salaries/players/'
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)
    a -=1
    url_list.append(new_url)

In [29]:
url_list[:5]

['https://hoopshype.com/salaries/players/2021-2022',
 'https://hoopshype.com/salaries/players/2020-2021',
 'https://hoopshype.com/salaries/players/2019-2020',
 'https://hoopshype.com/salaries/players/2018-2019',
 'https://hoopshype.com/salaries/players/2017-2018']

In [30]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:

    # Use Selenium, provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Make python pause for some time
    sleep(2)

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table', class_='hh-salaries-ranking-table hh-salaries-table-sortable responsive')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('td')[1:4]:
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Create a dataframe using the headers as column names
    p_salary = pd.DataFrame(columns = headers) 
    
    # Create a for loop to fill p_salary
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "").strip() for j in row_data] 
        length = len(p_salary)
        p_salary.loc[length] = row

    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(p_salary)


Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.
Completed: 6 webpage(s) scraped.
Completed: 7 webpage(s) scraped.
Completed: 8 webpage(s) scraped.
Completed: 9 webpage(s) scraped.
Completed: 10 webpage(s) scraped.
Completed: 11 webpage(s) scraped.
Completed: 12 webpage(s) scraped.
Completed: 13 webpage(s) scraped.
Completed: 14 webpage(s) scraped.
Completed: 15 webpage(s) scraped.
Completed: 16 webpage(s) scraped.
Completed: 17 webpage(s) scraped.
Completed: 18 webpage(s) scraped.
Completed: 19 webpage(s) scraped.
Completed: 20 webpage(s) scraped.
Completed: 21 webpage(s) scraped.
Completed: 22 webpage(s) scraped.
Completed: 23 webpage(s) scraped.
Completed: 24 webpage(s) scraped.
Completed: 25 webpage(s) scraped.
Completed: 26 webpage(s) scraped.


In [31]:
dataframe_list

[                Player      2021/22   2021/22(*)
 0        Stephen Curry  $45,780,966  $45,780,966
 1         James Harden  $44,310,840  $44,310,840
 2            John Wall  $44,310,840  $44,310,840
 3    Russell Westbrook  $44,211,146  $44,211,146
 4         Kevin Durant  $42,018,900  $42,018,900
 ..                 ...          ...          ...
 648     Anthony Barber      $53,176      $53,176
 649       Luca Vildoza      $42,789      $42,789
 650     Zavier Simpson      $37,223      $37,223
 651  Mfiondu Kabengele      $19,186      $19,186
 652     Melvin Frazier      $13,294      $13,294
 
 [653 rows x 3 columns],
                 Player      2020/21   2020/21(*)
 0        Stephen Curry  $43,006,362  $45,325,029
 1    Russell Westbrook  $41,358,814  $43,588,654
 2           Chris Paul  $41,358,814  $43,588,654
 3            John Wall  $41,254,920  $43,479,158
 4         James Harden  $41,254,920  $43,479,158
 ..                 ...          ...          ...
 573       Cam Reynolds

In [32]:
# Merge the dataframes
from functools import reduce
nba_salary = reduce(lambda l, r: pd.merge(l, r, on='Player', how='outer'), dataframe_list)
nba_salary

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$30,570,000","$32,960,751","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$19,169,800","$20,668,989","$18,063,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$44,211,146","$41,358,814","$43,588,654","$38,506,482","$40,844,595","$35,665,000","$38,454,209","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,018,900","$40,108,950","$42,271,404","$37,199,000","$39,457,722","$30,000,000","$32,346,173","$25,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$220,000","$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$205,200","$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$190,100","$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$184,100","$319,203"


In [33]:
# Save file, jic
nba_salary.to_csv('nba_salary.csv', encoding='utf-8', index=False)

In [34]:
nba_salary = pd.read_csv('nba_salary.csv', encoding='utf-8')
nba_salary

,Player,2021/22,2021/22(*),2020/21,2020/21(*),2019/20,2019/20(*),2018/19,2018/19(*),2017/18,...,2000/01,2000/01(*),1999/00,1999/00(*),1998/99,1998/99(*),1997/98,1997/98(*),1996/97,1996/97(*)
0,Stephen Curry,"$45,780,966","$45,780,966","$43,006,362","$45,325,029","$40,231,758","$42,674,629","$37,457,154","$40,386,520","$34,682,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$30,570,000","$32,960,751","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$44,310,840","$41,254,920","$43,479,158","$38,199,000","$40,518,442","$19,169,800","$20,668,989","$18,063,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$44,211,146","$41,358,814","$43,588,654","$38,506,482","$40,844,595","$35,665,000","$38,454,209","$28,299,399",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,018,900","$40,108,950","$42,271,404","$37,199,000","$39,457,722","$30,000,000","$32,346,173","$25,000,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$220,000","$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$205,200","$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$190,100","$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$184,100","$319,203"


## Clean and Prep Data

In [35]:
nba_salary.columns

Index(['Player', '2021/22', '2021/22(*)', '2020/21', '2020/21(*)', '2019/20',
       '2019/20(*)', '2018/19', '2018/19(*)', '2017/18', '2017/18(*)',
       '2016/17', '2016/17(*)', '2015/16', '2015/16(*)', '2014/15',
       '2014/15(*)', '2013/14', '2013/14(*)', '2012/13', '2012/13(*)',
       '2011/12', '2011/12(*)', '2010/11', '2010/11(*)', '2009/10',
       '2009/10(*)', '2008/09', '2008/09(*)', '2007/08', '2007/08(*)',
       '2006/07', '2006/07(*)', '2005/06', '2005/06(*)', '2004/05',
       '2004/05(*)', '2003/04', '2003/04(*)', '2002/03', '2002/03(*)',
       '2001/02', '2001/02(*)', '2000/01', '2000/01(*)', '1999/00',
       '1999/00(*)', '1998/99', '1998/99(*)', '1997/98', '1997/98(*)',
       '1996/97', '1996/97(*)'],
      dtype='object')

In [36]:
# Keep only columns that show current US Dollars adjusted for inflation, those columns with *
nba_salary = nba_salary.drop(['2021/22','2020/21', '2019/20', '2018/19', '2017/18',
                       '2016/17', '2015/16', '2014/15', '2013/14', '2012/13',
                       '2011/12', '2010/11', '2009/10', '2008/09', '2007/08',
                       '2006/07', '2005/06', '2004/05', '2003/04', '2002/03',
                       '2001/02', '2000/01', '1999/00', '1998/99', '1997/98',
                       '1996/97'], axis=1)

In [37]:
nba_salary

,Player,2021/22(*),2020/21(*),2019/20(*),2018/19(*),2017/18(*),2016/17(*),2015/16(*),2014/15(*),2013/14(*),...,2005/06(*),2004/05(*),2003/04(*),2002/03(*),2001/02(*),2000/01(*),1999/00(*),1998/99(*),1997/98(*),1996/97(*)
0,Stephen Curry,"$45,780,966","$45,325,029","$42,674,629","$40,386,520","$38,468,739","$13,654,081","$12,945,956","$12,116,632","$11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$43,479,158","$40,518,442","$32,960,751","$31,388,759","$29,918,259","$17,896,589","$16,750,126","$15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$43,479,158","$40,518,442","$20,668,989","$20,035,826","$17,761,997","$16,769,198","$15,618,561","$8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$43,588,654","$40,844,595","$38,454,209","$31,388,759","$29,918,259","$19,063,757","$17,918,740","$17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,271,404","$39,457,722","$32,346,173","$27,729,174","$29,918,259","$25,015,562","$22,795,904","$21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$319,203"


In [39]:
# Clean up the column names
nba_salary = nba_salary.rename(columns = {'2021/22(*)':'2021-22','2020/21(*)':'2020-21', 
                                    '2019/20(*)':'2019-20', '2018/19(*)':'2018-19', 
                                    '2017/18(*)':'2017-18','2016/17(*)':'2016-17', 
                                    '2015/16(*)':'2015-16', '2014/15(*)':'2014-15', 
                                    '2013/14(*)':'2013-14', '2012/13(*)':'2012-13',
                                    '2011/12(*)':'2011-12', '2010/11(*)':'2010-11', 
                                    '2009/10(*)':'2009-10', '2008/09(*)':'2008-09', 
                                    '2007/08(*)':'2007-08','2006/07(*)':'2006-07', 
                                    '2005/06(*)':'2005-06', '2004/05(*)':'2004-05', 
                                    '2003/04(*)':'2003-04', '2002/03(*)':'2002-03',
                                    '2001/02(*)':'2001-02', '2000/01(*)':'2000-01', 
                                    '1999/00(*)':'1999-00', '1998/99(*)':'1998-99', 
                                    '1997/98(*)':'1997-98','1996/97(*)':'1996-97'})

nba_salary

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,"$45,780,966","$45,325,029","$42,674,629","$40,386,520","$38,468,739","$13,654,081","$12,945,956","$12,116,632","$11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"$44,310,840","$43,479,158","$40,518,442","$32,960,751","$31,388,759","$29,918,259","$17,896,589","$16,750,126","$15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"$44,310,840","$43,479,158","$40,518,442","$20,668,989","$20,035,826","$17,761,997","$16,769,198","$15,618,561","$8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"$44,211,146","$43,588,654","$40,844,595","$38,454,209","$31,388,759","$29,918,259","$19,063,757","$17,918,740","$17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"$42,018,900","$42,271,404","$39,457,722","$32,346,173","$27,729,174","$29,918,259","$25,015,562","$22,795,904","$21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$319,203"


In [41]:
# Remove all '$' signs
nba_salary[nba_salary.columns] = nba_salary[nba_salary.columns].replace({'\$':''}, regex = True)
nba_salary

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,"45,780,966","45,325,029","42,674,629","40,386,520","38,468,739","13,654,081","12,945,956","12,116,632","11,504,868",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,"44,310,840","43,479,158","40,518,442","32,960,751","31,388,759","29,918,259","17,896,589","16,750,126","15,904,415",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,"44,310,840","43,479,158","40,518,442","20,668,989","20,035,826","17,761,997","16,769,198","15,618,561","8,680,071",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,"44,211,146","43,588,654","40,844,595","38,454,209","31,388,759","29,918,259","19,063,757","17,918,740","17,097,246",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,"42,018,900","42,271,404","39,457,722","32,346,173","27,729,174","29,918,259","25,015,562","22,795,904","21,843,723",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"381,449"
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"355,788"
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"329,606"
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"319,203"


In [42]:
# Remove all commas ','
nba_salary = nba_salary.replace(',', '', regex=True)
nba_salary

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,45780966,45325029,42674629,40386520,38468739,13654081,12945956,12116632,11504868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,44310840,43479158,40518442,32960751,31388759,29918259,17896589,16750126,15904415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,44310840,43479158,40518442,20668989,20035826,17761997,16769198,15618561,8680071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,44211146,43588654,40844595,38454209,31388759,29918259,19063757,17918740,17097246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,42018900,42271404,39457722,32346173,27729174,29918259,25015562,22795904,21843723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381449
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355788
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329606
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319203


In [43]:
nba_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2491 entries, 0 to 2490
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   2491 non-null   object
 1   2021-22  653 non-null    object
 2   2020-21  578 non-null    object
 3   2019-20  515 non-null    object
 4   2018-19  576 non-null    object
 5   2017-18  586 non-null    object
 6   2016-17  545 non-null    object
 7   2015-16  500 non-null    object
 8   2014-15  514 non-null    object
 9   2013-14  492 non-null    object
 10  2012-13  494 non-null    object
 11  2011-12  463 non-null    object
 12  2010-11  459 non-null    object
 13  2009-10  456 non-null    object
 14  2008-09  460 non-null    object
 15  2007-08  469 non-null    object
 16  2006-07  495 non-null    object
 17  2005-06  479 non-null    object
 18  2004-05  470 non-null    object
 19  2003-04  454 non-null    object
 20  2002-03  451 non-null    object
 21  2001-02  450 non-null    object
 22  

In [44]:
# Get column names
nba_salary.columns

Index(['Player', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97'],
      dtype='object')

In [45]:
# Change datatypes to numeric for appropriate columns
nba_salary[['2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97']] = nba_salary[['2021-22', '2020-21', '2019-20', '2018-19', '2017-18',
       '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12',
       '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06',
       '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00',
       '1998-99', '1997-98', '1996-97']].apply(pd.to_numeric)

In [46]:
nba_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2491 entries, 0 to 2490
Data columns (total 27 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   2491 non-null   object 
 1   2021-22  653 non-null    float64
 2   2020-21  578 non-null    float64
 3   2019-20  515 non-null    float64
 4   2018-19  576 non-null    float64
 5   2017-18  586 non-null    float64
 6   2016-17  545 non-null    float64
 7   2015-16  500 non-null    float64
 8   2014-15  514 non-null    float64
 9   2013-14  492 non-null    float64
 10  2012-13  494 non-null    float64
 11  2011-12  463 non-null    float64
 12  2010-11  459 non-null    float64
 13  2009-10  456 non-null    float64
 14  2008-09  460 non-null    float64
 15  2007-08  469 non-null    float64
 16  2006-07  495 non-null    float64
 17  2005-06  479 non-null    float64
 18  2004-05  470 non-null    float64
 19  2003-04  454 non-null    float64
 20  2002-03  451 non-null    float64
 21  2001-02  450 n

## Save File

In [47]:
# Save file to csv
nba_salary.to_csv('nba_salary2.csv', encoding='utf-8', index=False)

In [48]:
nba_salary = pd.read_csv('nba_salary2.csv', encoding='utf-8')
nba_salary

,Player,2021-22,2020-21,2019-20,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,...,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98,1996-97
0,Stephen Curry,45780966.0,45325029.0,42674629.0,40386520.0,38468739.0,13654081.0,12945956.0,12116632.0,11504868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,James Harden,44310840.0,43479158.0,40518442.0,32960751.0,31388759.0,29918259.0,17896589.0,16750126.0,15904415.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Wall,44310840.0,43479158.0,40518442.0,20668989.0,20035826.0,17761997.0,16769198.0,15618561.0,8680071.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Russell Westbrook,44211146.0,43588654.0,40844595.0,38454209.0,31388759.0,29918259.0,19063757.0,17918740.0,17097246.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Durant,42018900.0,42271404.0,39457722.0,32346173.0,27729174.0,29918259.0,25015562.0,22795904.0,21843723.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,Brett Szabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381449.0
2487,Earl Cureton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355788.0
2488,Kenny Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,329606.0
2489,Lloyd Daniels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319203.0


**Now all the columns that needed to be converted to numeric values have been converted!**

In [ ]:
# Find the average salary for each player
# Format average salary float to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format
nba_avg_sal = nba_sal.mean(axis=1)
nba_avg_sal

In [ ]:
# Add average salary column in dataframe
nba_sal['AVG SALARY'] = nba_sal.mean(axis=1)
nba_sal.sort_values(by='Player', ascending=True).tail(20)

In [ ]:
# We only need 2 columns, the player and avg salary columns
nba_salary = nba_sal[['Player', 'AVG SALARY']]
nba_salary

In [ ]:
# Save to csv file
nba_salary.to_csv('nba_salaries_cleaned.csv', encoding='utf-8', index=False)

### Merging nba_stats table with nba_salaries table

In [ ]:
# Import nba_stats
nba_stats = pd.read_csv('nba_stats_cleaned.csv')
nba_stats

In [ ]:
# Merge the two tables, nba_stats and nba_salary
nba_stats_sal = pd.merge(nba_salary, nba_stats.set_index('PLAYER'), left_on="Player", right_index=True)
nba_stats_sal

In [ ]:
# Change column 'Player' to 'PLAYER' for aesthetics
nba_stats_sal = nba_stats_sal.rename(columns = {'Player': 'PLAYER'})

nba_stats_sal

In [ ]:
# Save into csv file
nba_stats_sal.to_csv('nba_stats_merged.csv', encoding='utf-8', index=False)

In [ ]:
# Read file
nba_complete = pd.read_csv('nba_stats_merged.csv', encoding='utf-8')
nba_complete